# Task 2: Predicting customer buying behaviour
## 3. Data transformation

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import TargetEncoder

from sklearn.preprocessing import QuantileTransformer

from sklearn.preprocessing import MinMaxScaler

## Load the data

In [2]:
cat = pd.read_pickle("cat_eda.pickle")
num = pd.read_pickle("num_eda.pickle")

## Separate the target variable

In [3]:
target = num[["booking_complete"]].copy().reset_index(drop=True)

## CATEGORICAL VARIABLES TRANSFORMATION

In [4]:
cat.head()

,sales_channel,trip_type,flight_day,route,booking_origin
0,Internet,RoundTrip,Sat,OTHERS,New Zealand
1,Internet,RoundTrip,Sat,OTHERS,New Zealand
2,Internet,RoundTrip,Wed,OTHERS,India
3,Internet,RoundTrip,Sat,OTHERS,New Zealand
4,Internet,RoundTrip,Wed,OTHERS,India


### One Hot Encoding

#### Variables to apply OHE

In [5]:
var_ohe = ["sales_channel","trip_type"]

#### Instantiate

In [6]:
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

#### Train and apply

In [7]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Save as dataframe

In [8]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())
cat_ohe.head()

,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip
0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0


### Ordinal Encoding

#### Variables to apply OE

In [9]:
var_oe = ["flight_day"]

#### Order of those variables

In [10]:
order_flight_day = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]

#### Instantiate

In [11]:
oe = OrdinalEncoder(categories = [order_flight_day],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 10)

#### Fit and transform

In [12]:
cat_oe = oe.fit_transform(cat[var_oe]) + 1

#### Save as a dataframe

In [13]:
#Add suffixes to variables' names
names_oe = [variable + '_oe' for variable in var_oe]

#Save as a dataframe
cat_oe = pd.DataFrame(cat_oe, columns = names_oe)
cat_oe.head()

,flight_day_oe
0,6.0
1,6.0
2,3.0
3,6.0
4,3.0


### Target Encoding

#### Variables to apply TE

In [14]:
var_te = ["route","booking_origin"]

#### Instantiate

In [15]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Fit and transform

In [16]:
cat_te = te.fit_transform(cat[var_te].reset_index(drop=True), y = target)

#### Save as a dataframe

In [17]:
#Add suffixes to variables' names
names_te = [variable + '_te' for variable in var_te]

#Save as a dataframe
cat_te = pd.DataFrame(cat_te, columns = names_te)
cat_te.head()

,route_te,booking_origin_te
0,0.149552,0.050000
1,0.149552,0.050000
2,0.149552,0.101749
3,0.149552,0.050000
4,0.149552,0.101749


## NUMERICAL VARIABLES TRANSFORMATION

### Normalization (Gauss)

In [18]:
var_qt = ["num_passengers","purchase_lead","length_of_stay","flight_hour","flight_duration"]

##### Instantiate

In [19]:
qt = QuantileTransformer(output_distribution='normal')

##### Fit and transform

In [20]:
num_qt = qt.fit_transform(num[var_qt])

##### Save as a dataframe

In [21]:
#Add suffixes to variables' names
names_qt = [variable + '_qt' for variable in var_qt]

#Save as a dataframe
num_qt = pd.DataFrame(num_qt,columns = names_qt)

## COMBINE ALL DATASETS

First let's add the non-transformed data

In [22]:
variables = cat.columns.to_list() + num.iloc[:,:-1].columns.to_list()
var_trans = var_oe + var_ohe + var_te + var_qt
var_nt = [name for name in variables if name not in var_trans]
var_nt

['wants_extra_baggage', 'wants_preferred_seat', 'wants_in_flight_meals']

In [23]:
num_nt = num[var_nt].reset_index().iloc[:,1:]

In [24]:
dataframes = []
dataframes.extend(value for name, value in locals().items() if name.startswith('cat_') or name.startswith('num_'))

### Concatenate all the datasets

In [25]:
df = pd.concat(dataframes, axis = 1)
df

,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip,flight_day_oe,route_te,booking_origin_te,num_passengers_qt,purchase_lead_qt,length_of_stay_qt,flight_hour_qt,flight_duration_qt,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals
0,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,0.703922,1.517545,0.157469,-0.288750,-1.037937,1,0,0
1,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,-5.199338,0.645631,0.218773,-1.018778,-1.037937,0,0,0
2,1.0,0.0,0.0,0.0,1.0,3.0,0.149552,0.101749,0.703922,1.397837,0.366873,1.475269,-1.037937,1,1,0
3,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,-5.199338,0.514923,0.814401,-0.809168,-1.037937,0,0,1
4,1.0,0.0,0.0,0.0,1.0,3.0,0.149552,0.101749,0.703922,0.218773,0.366873,1.111702,-1.037937,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49276,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050591,0.703922,-0.502074,-0.213636,0.022584,-0.793602,1,0,1
49277,1.0,0.0,0.0,0.0,1.0,7.0,0.149552,0.050591,-5.199338,0.636387,-0.213636,-0.809168,-0.793602,0,0,0
49278,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050591,-5.199338,-0.590949,-0.213636,1.934489,-0.793602,0,0,1
49279,1.0,0.0,0.0,0.0,1.0,1.0,0.149552,0.050591,-5.199338,-0.922330,-0.213636,0.330873,-0.793602,1,0,1


## FEATURE SCALING

### Min-Max scaling (Normalization between 0 and 1)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49281 entries, 0 to 49280
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sales_channel_Internet  49281 non-null  float64
 1   sales_channel_Mobile    49281 non-null  float64
 2   trip_type_CircleTrip    49281 non-null  float64
 3   trip_type_OneWay        49281 non-null  float64
 4   trip_type_RoundTrip     49281 non-null  float64
 5   flight_day_oe           49281 non-null  float64
 6   route_te                49281 non-null  float64
 7   booking_origin_te       49281 non-null  float64
 8   num_passengers_qt       49281 non-null  float64
 9   purchase_lead_qt        49281 non-null  float64
 10  length_of_stay_qt       49281 non-null  float64
 11  flight_hour_qt          49281 non-null  float64
 12  flight_duration_qt      49281 non-null  float64
 13  wants_extra_baggage     49281 non-null  int64  
 14  wants_preferred_seat    49281 non-null

In [27]:
var_mms = df.iloc[:,5:12].columns
var_mms

Index(['flight_day_oe', 'route_te', 'booking_origin_te', 'num_passengers_qt',
       'purchase_lead_qt', 'length_of_stay_qt', 'flight_hour_qt'],
      dtype='object')

#### Instantiate

In [28]:
mms = MinMaxScaler()

#### Fit and transform

In [29]:
df_mms = mms.fit_transform(df[var_mms])

#### Save as a dataframe

In [30]:
#Add suffixes to variables' names
nombres_mms = [variable + '_mms' for variable in var_mms]

#Save as a dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

### COMBINE SCALED DATASETS

In [31]:
include = [df, df_mms, target]

In [32]:
df_final = pd.concat(include, axis=1)
df_final.head()

,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip,flight_day_oe,route_te,booking_origin_te,num_passengers_qt,purchase_lead_qt,...,wants_preferred_seat,wants_in_flight_meals,flight_day_oe_mms,route_te_mms,booking_origin_te_mms,num_passengers_qt_mms,purchase_lead_qt_mms,length_of_stay_qt_mms,flight_hour_qt_mms,booking_complete
0,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,0.703922,1.517545,...,0,0,0.833333,0.32154,0.000000,0.567693,0.645936,0.515143,0.472232,0
1,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,-5.199338,0.645631,...,0,0,0.833333,0.32154,0.000000,0.000000,0.562088,0.521039,0.402028,0
2,1.0,0.0,0.0,0.0,1.0,3.0,0.149552,0.101749,0.703922,1.397837,...,1,0,0.333333,0.32154,0.175418,0.567693,0.634424,0.535281,0.641871,0
3,1.0,0.0,0.0,0.0,1.0,6.0,0.149552,0.050000,-5.199338,0.514923,...,0,1,0.833333,0.32154,0.000000,0.000000,0.549518,0.578318,0.422185,0
4,1.0,0.0,0.0,0.0,1.0,3.0,0.149552,0.101749,0.703922,0.218773,...,0,1,0.333333,0.32154,0.175418,0.567693,0.521039,0.535281,0.606908,0


Save the final dataframe in .pickle format to proceed to the next step (modeling).

In [33]:
pd.to_pickle(df_final, "df_transformed.pickle")